#  TP 2: 

### 18 mars 2024

### Par Samuel Fortin, Philippe Truchon et Benjamin Trudel

## TP2.1 Décomposition QR par la méthode Householder

#### Fonctions générales

In [47]:
import fnmatch
import functools
import os

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import math


sns.set_theme(style="ticks", palette="deep")

plt.rcParams["axes.spines.right"] = False
plt.rcParams["axes.spines.top"] = False



# Liste les noms de fichier d'un dossier
def listNameOfFiles(directory: str, extension="csv"):
    found_files = []
    for file in os.listdir(directory):
        if fnmatch.fnmatch(file, f"*.{extension}"):
            found_files.append(file)
    return found_files


# Lis et crée une matrice numpy à partir de chemin d'un fichier texte
def readTXT(path: str):
    fich = open(path, "r")
    fich_str = list(fich)
    fich.close()
    x = []
    for i in fich_str:
        elem_str = i.replace("\n", "")
        x.append(float(elem_str))
    return np.array(x)

sign = lambda x: math.copysign(1, x) # two will work

path = os.path.abspath("")
files_name = listNameOfFiles(path)


### Questions:

### a)

<i>À l’aide des équations (2.1.2) et (2.1.3), démontrez que les matrices de réflexion Q_i sont orthogonales.</i>

### b)

<i>Démontrez l’équation (2.1.5) et que la matrice Q est orthogonale.</i>

 

### c)

<i>Implémentez la fonction householder_qr qui prend en argument une matrice A et qui retourne les matrices Q
et R obtenues par la méthode de Householder.</i>


In [48]:
def householder_qr(A: np.ndarray):

    m = np.shape(A)[0]
    n = np.shape(A)[1]
    R = A.copy()
    x = A[:,0]
    e1 = np.zeros(m).T
    e1[0] = 1
    v = (sign(x[0])*np.abs(x[1]) * e1)+x # Je crois que l'erreur est ici ou certaine opération doivent ce faire avec une copie de A
    H = np.identity(m)
    H[0:, 0:] -= -2*(v @ v.T)/(v.T @ v)
    
    Q = np.identity(m)
    Q = H @ Q
    Q_i = [Q]

    for i in range(1, n):
        s_0 = Q_i[0]
        for s in range(1, len(Q_i)):
            Q_it = Q_i[s] @ s_0
        try:
            x = Q_it @ A
            x = x[:,i][-(m-i):]
        except UnboundLocalError:
            Q_it = s_0
            x = Q_it @ A  
            x = x[:,i][-(m-i):]
        s_0 = Q_it


        v = (sign(x[0])*np.abs(x[1]))+x
        v[0] = 1
        H = np.identity(m)
        H[i:, i:] -= -2*(v @ v.T)/(v.T @ v)


        Q = np.identity(m)
        Q = H @ Q
        R = H @ R
        Q_i.append(np.array(Q))

    s_0 = Q_i[0]
    for s in range(1, len(Q_i)):
        Q_it = Q_i[s-1] @ s_0
        s_0 = Q_it

    return Q_it, R
        
r1 = householder_qr(np.arange(12).reshape(4,3))
r2 = np.linalg.qr(np.arange(12).reshape(4,3), mode='complete')
print(r1)
print(r2)

(array([[ 21.,  20.,  20.,  20.],
       [140., 143., 142., 142.],
       [140., 142., 143., 142.],
       [140., 142., 142., 143.]]), array([[  0.,   1.,   2.],
       [ 39.,  46.,  53.],
       [216., 251., 286.],
       [219., 254., 289.]]))
QRResult(Q=array([[ 0.        , -0.83666003,  0.5301023 , -0.13780983],
       [-0.26726124, -0.47809144, -0.60408569,  0.57886137],
       [-0.53452248, -0.11952286, -0.38213553, -0.74429324],
       [-0.80178373,  0.23904572,  0.45611892,  0.30324171]]), R=array([[-1.12249722e+01, -1.28285396e+01, -1.44321071e+01],
       [ 0.00000000e+00, -1.19522861e+00, -2.39045722e+00],
       [ 0.00000000e+00,  0.00000000e+00,  3.48440273e-15],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00]]))


In [56]:
def qr(A):
    m, n = A.shape
    Q = np.eye(m)
    for i in range(n - (m == n)):
        H = np.eye(m)
        x = A[i:, i]
        v = x / (x[0] + np.copysign(np.linalg.norm(x), x[0]))
        v[0] = 1
        Ht = np.eye(x.shape[0])
        Ht -= (2 * np.dot(v[:, None], v[None, :])/np.dot(v, v))
        H[i:, i:] = Ht
        Q = np.dot(Q, H)
        A = np.dot(H, A)
    return Q, A


# task 1: show qr decomp of wp example
a = np.arange(12).reshape(4,3)

q, r = qr(a)
print('q:\n', q.round(6))
print('r:\n', r.round(6))
r2 = np.linalg.qr(np.arange(12).reshape(4,3), mode='complete')
print(r2)


q:
 [[ 0.       -0.83666  -0.239797 -0.49244 ]
 [-0.267261 -0.478091  0.686772  0.477853]
 [-0.534522 -0.119523 -0.654154  0.521615]
 [-0.801784  0.239046  0.207179 -0.507028]]
r:
 [[-11.224972 -12.82854  -14.432107]
 [ -0.        -1.195229  -2.390457]
 [ -0.        -0.         0.      ]
 [ -0.        -0.        -0.      ]]
QRResult(Q=array([[ 0.        , -0.83666003,  0.5301023 , -0.13780983],
       [-0.26726124, -0.47809144, -0.60408569,  0.57886137],
       [-0.53452248, -0.11952286, -0.38213553, -0.74429324],
       [-0.80178373,  0.23904572,  0.45611892,  0.30324171]]), R=array([[-1.12249722e+01, -1.28285396e+01, -1.44321071e+01],
       [ 0.00000000e+00, -1.19522861e+00, -2.39045722e+00],
       [ 0.00000000e+00,  0.00000000e+00,  3.48440273e-15],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00]]))


### d)

<i>À l’aide d’une matrice de dimension 4 × 3 de votre choix, testez votre fonction householder_qr et comparez les
résultats obtenus avec ceux obtenus à l’aide de la fonction numpy.linalg.qr. Les matrices sont-elles exactement
les mêmes ? Si non, est-ce un problème?</i>



### e)

<i>À l’aide de la matrice utilisée en d, illustrez comment la multiplication successive des matrices Q_i triangularise
progressivement la matrice A. Dans l’élan, assurez-vous que les matrices Q et R obtenues sont bien orthogonale et
triangulaire supérieure, respectivement.</i>
